# Building an Approximate Nearest Neighbours Index

This tutorial shows how to build an approximate nearest neighbours (ann) index for a given set of embeddings.

We use the Spotify [ANNOY](https://github.com/spotify/annoy) library for this task.

The following are the steps of this tutorial:
1. Build the annoy index given the embeddings saved in the TSV file
2. Load movie information form the downloaded data
3. Use the index to find similar movies to a given one

<a href="https://colab.research.google.com/github.com/ksalama/data2cooc2emb2ann/blob/master/track2ann/03-Building_an_Approximate_Nearest_Neighbours_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [1]:
# !pip install -r ../requirements.txt

In [2]:
import os
from annoy import AnnoyIndex
from datetime import datetime
import pandas as pd

In [3]:
WORKSPACE = './workspace'
DATA_DIR = '{}/data'.format(WORKSPACE)
DATASET = 'ml-1m'
ratings_data_file = os.path.join(DATA_DIR, '{}/ratings.dat'.format(DATASET))
movies_data_file = os.path.join(DATA_DIR, '{}/movies.dat'.format(DATASET))
embeddings_file_path = os.path.join(WORKSPACE,'embeddings.tsv')
index_file_path = os.path.join(WORKSPACE,'embed-ann.index')

## 1. Build Annoy Index

In [4]:
def build_embeddings_index(embeddings_file, embedding_size, num_trees):
    
    annoy_index = AnnoyIndex(embedding_size, metric='angular')
    idx2item_mapping = dict()
    item2idx_mapping = dict()
    
    idx = 0
    
    with open(embeddings_file_path) as embedding_file:
        
        while True:
            line = embedding_file.readline()
            if not line: break
                
            parts = line.split('\t')
            item_id = parts[0]
            embedding = [float(v) for v in parts[1:]]
            
            idx2item_mapping[idx] = item_id
            item2idx_mapping[item_id] = idx

            annoy_index.add_item(idx, embedding)
            idx+=1
        
    print("{} items where added to the index".format(idx))
    annoy_index.build(n_trees=num_trees)
    print("Index is built")
    return annoy_index, idx2item_mapping, item2idx_mapping


In [5]:
num_trees = 1000
embedding_size = 32

index, idx2item_mapping,  item2idx_mapping = build_embeddings_index(
    embeddings_file_path, embedding_size, num_trees)

3706 items where added to the index
Index is built


## 2. Load movie data

In [6]:
ratings_header = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings_data = pd.read_csv(ratings_data_file, sep="::", names=ratings_header)
ratings_data.head()

/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
movies_header = ['movie_id', 'title', 'genres']
movies_data = pd.read_csv(movies_data_file, sep="::", names=movies_header, encoding = "ISO-8859-1")
movies_data.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### Top frequent movies

In [9]:
frequent_movie_ids = list(ratings_data.movie_id.value_counts().index[:30])
movies_data[movies_data['movie_id'].isin(frequent_movie_ids)]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
108,110,Braveheart (1995),Action|Drama|War
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
293,296,Pulp Fiction (1994),Crime|Drama
315,318,"Shawshank Redemption, The (1994)",Drama
352,356,Forrest Gump (1994),Comedy|Romance|War
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
523,527,Schindler's List (1993),Drama|War
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
589,593,"Silence of the Lambs, The (1991)",Drama|Thriller


## 3. Find similar items

In [22]:
def get_similar_items(item_id, num_matches=10):
    
    idx = item2idx_mapping[item_id]
    
    similar_idx = index.get_nns_by_item(
        idx, num_matches, search_k=-1, include_distances=False)
    
    similar_item_ids = []
    for idx in similar_idx:
        similar_item_ids.append(idx2item_mapping[idx])
    
    return similar_item_ids

In [38]:
# movies_data['movie_id'][similar_item_ids[1]]

KeyError: '3481'

In [43]:
for movie_id in sorted(frequent_movie_ids):
    movie_title = movies_data[movies_data['movie_id'] == movie_id].title.values[0]
    print("Movie: {}".format(movie_title))
    similar_item_ids = get_similar_items(str(movie_id))
    similar_item_ids = list(map(int, similar_item_ids))
    
    similar_movies = movies_data[movies_data['movie_id'].isin(similar_item_ids)].title
    print("Similar Movies:")
    print(similar_movies)
    print("--------------------------------------")
                             

Movie: Toy Story (1995)
Similar Movies:
0                        Toy Story (1995)
148                      Apollo 13 (1995)
584                        Aladdin (1992)
941          It's a Wonderful Life (1946)
1081    E.T. the Extra-Terrestrial (1982)
1179           Princess Bride, The (1987)
1250            Back to the Future (1985)
2286                 Bug's Life, A (1998)
2728                           Big (1988)
3045                   Toy Story 2 (1999)
Name: title, dtype: object
--------------------------------------
Movie: Braveheart (1995)
Similar Movies:
108                             Braveheart (1995)
315              Shawshank Redemption, The (1994)
453                          Fugitive, The (1993)
585             Terminator 2: Judgment Day (1991)
1023                              Die Hard (1988)
1220                       Terminator, The (1984)
1271    Indiana Jones and the Last Crusade (1989)
1959                   Saving Private Ryan (1998)
2693                      Sixth S